In [23]:
import os
os.system("sudo pip install findspark sql_magic")

0

In [1]:
import findspark
findspark.init('/usr/lib/spark')

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType

In [4]:
spark = SparkSession\
    .builder\
    .getOrCreate()

In [5]:
%load_ext sql_magic
%config SQL.conn_name = 'spark'

In [6]:
df = spark.read.csv("s3n://2017edmfasatb/nypd_complaints/data/NYPD_Complaint_Data_Historic.csv", header = True, inferSchema = True)

In [7]:
date_func =  udf(lambda x: datetime.strptime(x, '%m/%d/%Y'), DateType())
time_func =  udf(lambda x: datetime.strptime(x, '%H:%M:%S'), DateType())

df = df.fillna('01/01/1900', subset = ['CMPLNT_FR_DT', 'CMPLNT_TO_DT', 'RPT_DT'])
df = df.fillna('00:00:00', subset = ['CMPLNT_FR_TM', 'CMPLNT_TO_TM'])

df = df.withColumn('CMPLNT_FR_DT_FORMATTED', date_func(col('CMPLNT_FR_DT')))
df = df.withColumn('CMPLNT_TO_DT_FORMATTED', date_func(col('CMPLNT_TO_DT')))
df = df.withColumn('RPT_DT_FORMATTED', date_func(col('RPT_DT')))
df = df.withColumn('CMPLNT_FR_TM_FORMATTED', time_func(col('CMPLNT_FR_TM')))
df = df.withColumn('CMPLNT_TO_TM_FORMATTED', time_func(col('CMPLNT_TO_TM')))

df.printSchema()

root
 |-- CMPLNT_NUM: integer (nullable = true)
 |-- CMPLNT_FR_DT: string (nullable = false)
 |-- CMPLNT_FR_TM: string (nullable = false)
 |-- CMPLNT_TO_DT: string (nullable = false)
 |-- CMPLNT_TO_TM: string (nullable = false)
 |-- RPT_DT: string (nullable = false)
 |-- KY_CD: integer (nullable = true)
 |-- OFNS_DESC: string (nullable = true)
 |-- PD_CD: integer (nullable = true)
 |-- PD_DESC: string (nullable = true)
 |-- CRM_ATPT_CPTD_CD: string (nullable = true)
 |-- LAW_CAT_CD: string (nullable = true)
 |-- JURIS_DESC: string (nullable = true)
 |-- BORO_NM: string (nullable = true)
 |-- ADDR_PCT_CD: integer (nullable = true)
 |-- LOC_OF_OCCUR_DESC: string (nullable = true)
 |-- PREM_TYP_DESC: string (nullable = true)
 |-- PARKS_NM: string (nullable = true)
 |-- HADEVELOPT: string (nullable = true)
 |-- X_COORD_CD: integer (nullable = true)
 |-- Y_COORD_CD: integer (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Lat_Lon: strin

In [8]:
extract_year =  udf(lambda x: x.year)
extract_month =  udf(lambda x: x.month)

df = df.withColumn('CMPLNT_FR_DT_YEAR', extract_year(col('CMPLNT_FR_DT_FORMATTED')))
df = df.withColumn('CMPLNT_FR_DT_MONTH', extract_month(col('CMPLNT_FR_DT_FORMATTED')))

In [9]:
df.createOrReplaceTempView("complaint")

In [18]:
result = %read_sql SELECT * FROM complaint LIMIT 10;
result

Query started at 07:43:37 PM UTC; Query executed in 0.73 m

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,...,Latitude,Longitude,Lat_Lon,CMPLNT_FR_DT_FORMATTED,CMPLNT_TO_DT_FORMATTED,RPT_DT_FORMATTED,CMPLNT_FR_TM_FORMATTED,CMPLNT_TO_TM_FORMATTED,CMPLNT_FR_DT_YEAR,CMPLNT_FR_DT_MONTH
0,101109527,12/31/2015,23:45:00,01/01/1900,00:00:00,12/31/2015,113,FORGERY,729.0,"FORGERY,ETC.,UNCLASSIFIED-FELO",...,40.828848,-73.916661,"(40.828848333, -73.916661142)",2015-12-31,1900-01-01,2015-12-31,1900-01-01,1900-01-01,2015,12
1,153401121,12/31/2015,23:36:00,01/01/1900,00:00:00,12/31/2015,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,None,...,40.697338,-73.784557,"(40.697338138, -73.784556739)",2015-12-31,1900-01-01,2015-12-31,1900-01-01,1900-01-01,2015,12
2,569369778,12/31/2015,23:30:00,01/01/1900,00:00:00,12/31/2015,117,DANGEROUS DRUGS,503.0,"CONTROLLED SUBSTANCE,INTENT TO",...,40.802607,-73.945052,"(40.802606608, -73.945051911)",2015-12-31,1900-01-01,2015-12-31,1900-01-01,1900-01-01,2015,12
3,968417082,12/31/2015,23:30:00,01/01/1900,00:00:00,12/31/2015,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,...,40.654549,-73.726339,"(40.654549444, -73.726338791)",2015-12-31,1900-01-01,2015-12-31,1900-01-01,1900-01-01,2015,12
4,641637920,12/31/2015,23:25:00,12/31/2015,23:30:00,12/31/2015,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,...,40.738002,-73.987891,"(40.7380024, -73.98789129)",2015-12-31,2015-12-31,2015-12-31,1900-01-01,1900-01-01,2015,12
5,365661343,12/31/2015,23:18:00,12/31/2015,23:25:00,12/31/2015,106,FELONY ASSAULT,109.0,"ASSAULT 2,1,UNCLASSIFIED",...,40.665023,-73.957111,"(40.665022689, -73.957110763)",2015-12-31,2015-12-31,2015-12-31,1900-01-01,1900-01-01,2015,12
6,608231454,12/31/2015,23:15:00,01/01/1900,00:00:00,12/31/2015,235,DANGEROUS DRUGS,511.0,"CONTROLLED SUBSTANCE, POSSESSI",...,40.720200,-73.988735,"(40.720199996, -73.988735082)",2015-12-31,1900-01-01,2015-12-31,1900-01-01,1900-01-01,2015,12
7,265023856,12/31/2015,23:15:00,12/31/2015,23:15:00,12/31/2015,118,DANGEROUS WEAPONS,792.0,WEAPONS POSSESSION 1 & 2,...,40.845707,-73.910398,"(40.845707148, -73.910398033)",2015-12-31,2015-12-31,2015-12-31,1900-01-01,1900-01-01,2015,12
8,989238731,12/31/2015,23:15:00,12/31/2015,23:30:00,12/31/2015,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,...,40.856711,-73.891900,"(40.856711291, -73.891899956)",2015-12-31,2015-12-31,2015-12-31,1900-01-01,1900-01-01,2015,12
9,415095955,12/31/2015,23:10:00,12/31/2015,23:10:00,12/31/2015,341,PETIT LARCENY,338.0,"LARCENY,PETIT FROM BUILDING,UN",...,40.765618,-73.963623,"(40.765617688, -73.96362342)",2015-12-31,2015-12-31,2015-12-31,1900-01-01,1900-01-01,2015,12
